In [42]:
import pandas as pd
import matplotlib as plt
import numpy as np
import requests
import os
from dotenv import load_dotenv
import time
import re


In [45]:

load_dotenv()
API_key = os.getenv('ALPHAVANTAGE_API_KEY')

symbols = ["AAPL", "MSFT", "SPY", "TSLA", "GOOGL", "AMZN", "META"]

base_url = "https://www.alphavantage.co/query"

all_data = []  # List to store each symbol's data

for symbol in symbols:
    print(f"Downloading weekly data for {symbol}...")

    params = {
        "function": "TIME_SERIES_WEEKLY_ADJUSTED",
        "symbol": symbol,
        "apikey": API_key
    }

    for attempt in range(3):
        r = requests.get(base_url, params=params)
        try:
            data = r.json()
            break
        except ValueError:
            print(f"Invalid response for {symbol}, attempt {attempt+1}")
            print("Raw response:", r.text[:200])
            time.sleep(15)
    else:
        print(f"Failed to get valid JSON for {symbol}, skipping...")
        continue

    if "Weekly Adjusted Time Series" not in data:
        print(f"Error for {symbol}: {data}")
        time.sleep(15)
        continue

    df = pd.DataFrame(data["Weekly Adjusted Time Series"]).T

    # Keep default integer index, create new column 'Date'
    df['Date'] = pd.to_datetime(df.index)

    # Sort by Date if needed
    df = df.sort_values('Date')

    df = df[df['Date'] >= (pd.Timestamp.today() - pd.DateOffset(years=5))]

    df['Symbol'] = symbol
    df.reset_index(drop=True, inplace=True)  # Drop old string index

    all_data.append(df)
    time.sleep(15)


Error for AAPL: {'Information': 'We have detected your API key as 82FA7IZU56NKJPN5 and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
Error for MSFT: {'Information': 'We have detected your API key as 82FA7IZU56NKJPN5 and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
Error for SPY: {'Information': 'We have detected your API key as 82FA7IZU56NKJPN5 and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
Error for TSLA: {'Information': 'We have detected your API key as 82FA7IZU56NKJPN5 and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https:/

In [47]:
if all_data:
    df = pd.concat(all_data)

    # Clean column names (remove numbers like "1. open")
    df.columns = [re.sub(r'^\d+\.\s*', '', col) for col in df.columns]

    # Reorder columns
    df = df[['Symbol', 'Date'] + [col for col in df.columns if col not in ['Symbol', 'Date']]]

    #df.to_csv("weekly_5yr_all_symbols.csv", index=True)  # Keep integer index in CSV

In [50]:
df['close'] = df['close'].astype(float)

# Sort by Symbol and Date to make sure diff works correctly
df = df.sort_values(['Symbol', 'Date'])

# Create a new column with difference from previous close
df['Close_diff'] = df.groupby('Symbol')['close'].diff()

In [ ]:
df.head()

,Symbol,Date,open,high,low,close,adjusted close,volume,dividend amount,Close_diff
0,AMZN,2020-11-27,3116.6986,3216.1900,3065.4600,3195.34,159.7670,14463046,0.0000,NaN
1,AMZN,2020-12-04,3208.4800,3248.9500,3125.5500,3162.58,158.1290,17492424,0.0000,-32.76
2,AMZN,2020-12-11,3156.4800,3184.1300,3072.8200,3116.42,155.8210,16233336,0.0000,-46.16
3,AMZN,2020-12-18,3143.0000,3263.5100,3126.0000,3201.65,160.0825,21377156,0.0000,85.23
4,AMZN,2020-12-24,3200.0100,3226.9666,3166.0000,3172.69,158.6345,9764331,0.0000,-28.96
5,AMZN,2020-12-31,3194.0000,3350.6500,3172.6900,3256.93,162.8465,16660177,0.0000,84.24
6,AMZN,2021-01-08,3270.0000,3272.0000,3131.1600,3182.70,159.1350,18514101,0.0000,-74.23
7,AMZN,2021-01-15,3148.0100,3189.9500,3086.0000,3104.25,155.2125,17723802,0.0000,-78.45
8,AMZN,2021-01-22,3107.0000,3348.5500,3096.0000,3292.23,164.6115,16381942,0.0000,187.98
9,AMZN,2021-01-29,3328.5000,3363.8900,3184.5500,3206.20,160.3100,18807935,0.0000,-86.03
